# EX3 : Proof Of Stake VS Proof of Work

### Idriss Khattabi

In [1]:
import hashlib, time, random

### Block Class :

In [2]:
class Block:
    def __init__(self, index, previous_hash, data):
        self.index = index
        self.timestamp = time.time()
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = 0 
        self.hash = self.calculate_hash()

    # calculate the block's hash
    def calculate_hash(self):
        block_info = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}{self.nonce}".encode()
        return hashlib.sha256(block_info).hexdigest()

    def __str__(self):
        return f"Block {self.index} [Hash: {self.hash}, Data: {self.data}, Previous Hash: {self.previous_hash}, Nonce: {self.nonce}]"

### Proof Of Work :

In [4]:
class PoW:
    def __init__(self, difficulty):
        self.chain = [self.create_genesis_block()]
        self.difficulty = difficulty

    def create_genesis_block(self):
        return Block(0, "0", "First Block")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = self.mine_block(new_block)
        self.chain.append(new_block)

    def mine_block(self, block):
        """Implements the PoW algorithm by repeatedly calculating the block's hash until it matches the difficulty target"""
        
        target = "0" * self.difficulty
        start_time = time.time()
        print(f"Mining block with difficulty {self.difficulty}...")

        while not block.hash.startswith(target):
            block.nonce += 1
            block.hash = block.calculate_hash()

        end_time = time.time()
        print(f"Block mined: {block.hash}")
        print(f"Time taken for PoW: {end_time - start_time} seconds\n")
        return block.hash

In **Proof of Work (PoW)**, participants (miners) compete to solve a difficult puzzle that requires finding a "nonce" — a number which, when combined with block data, creates a hash (a unique digital fingerprint) that meets a specific pattern, often starting with a certain number of zeros. This pattern defines the "difficulty level," and the higher it is, the harder the puzzle.

**The process**:

- Miners try different nonces to generate a hash matching the required pattern.
- Once a miner finds the correct hash, the solution is easily verified and added to the blockchain.
- This mined block becomes part of the chain, and the process restarts for the next block.

---
### Proof Of Stake :

In [5]:
class Validator:
    def __init__(self, name, stake):
        self.name = name
        self.stake = stake

In [6]:
class PoS:
    def __init__(self, validators):
        self.chain = [self.create_genesis_block()]
        self.validators = validators

    def create_genesis_block(self):
        return Block(0, "0", "First Block")

    def get_latest_block(self):
        return self.chain[-1]

    def select_validator(self):
        """Selects a validator based on their stake.
        A random number between 0 and the total stake is generated,
        and validators are sequentially chosen based on their stake proportion."""
        
        total_stake = sum([validator.stake for validator in self.validators])
        rand = random.uniform(0, total_stake)
        current = 0
        for validator in self.validators:
            current += validator.stake
            if current > rand:
                return validator.name

    def add_block(self, new_block):
        start_time = time.time()
        selected_validator = self.select_validator()
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = new_block.calculate_hash()

        end_time = time.time()
        print(f"Block validated by {selected_validator} with PoS")
        print(f"Time taken for PoS: {end_time - start_time} seconds\n")
        self.chain.append(new_block)

In **Proof of Stake (PoS)**, participants (validators) are selected to add new blocks to the blockchain based on the amount of cryptocurrency they hold, known as their "stake." Unlike Proof of Work (PoW), PoS does not require heavy computations, making it more energy-efficient.

**Process in Brief**:
- Validators: Each validator holds a stake, representing their commitment to the network.
- Selection: A validator is selected based on their stake proportion. In this code, a random value is generated, and validators are chosen according to their stake — higher stake means a higher probability of being chosen.
- Block Addition: The chosen validator validates the new block, calculating its hash based on previous data. The block is then added to the blockchain.

---
### Test :

In [9]:
def compare_pow_and_pos():

    print("===== Proof of Work (PoW) =====")
    pow_blockchain = PoW(difficulty=4)
    pow_blockchain.add_block(Block(1, pow_blockchain.get_latest_block().hash, "This is Block 1 - PoW"))
    pow_blockchain.add_block(Block(2, pow_blockchain.get_latest_block().hash, "This is Block 2 - PoW"))
    for block in pow_blockchain.chain :
        print(block)

    print("\n===== Proof of Stake (PoS) =====")
    validators = [
        Validator("Idriss", stake=10),
        Validator("Mohammed", stake=20),
        Validator("Hiba", stake=30)
    ]
    pos_blockchain = PoS(validators=validators)
    pos_blockchain.add_block(Block(1, pos_blockchain.get_latest_block().hash, "This is Block 1 - PoS"))
    pos_blockchain.add_block(Block(2, pos_blockchain.get_latest_block().hash, "This is Block 2 - PoS"))
    for block in pos_blockchain.chain :
        print(block)

In [10]:
compare_pow_and_pos()

===== Proof of Work (PoW) =====
Mining block with difficulty 4...
Block mined: 00003e3c52517746588928917294f9687a7ffe8af43924c1a4ff7ff4c3249c57
Time taken for PoW: 0.16894173622131348 seconds

Mining block with difficulty 4...
Block mined: 000040aead2d1fd326493a3a779541b10b6397c606ead0d7b041e5c7efd093b7
Time taken for PoW: 0.1119987964630127 seconds

Block 0 [Hash: 1a75dfa6e46131a42cff568505393ef43181b4ab297bbc230552ec10bf3fce2b, Data: First Block, Previous Hash: 0, Nonce: 0]
Block 1 [Hash: 00003e3c52517746588928917294f9687a7ffe8af43924c1a4ff7ff4c3249c57, Data: This is Block 1 - PoW, Previous Hash: 1a75dfa6e46131a42cff568505393ef43181b4ab297bbc230552ec10bf3fce2b, Nonce: 61731]
Block 2 [Hash: 000040aead2d1fd326493a3a779541b10b6397c606ead0d7b041e5c7efd093b7, Data: This is Block 2 - PoW, Previous Hash: 00003e3c52517746588928917294f9687a7ffe8af43924c1a4ff7ff4c3249c57, Nonce: 41522]

===== Proof of Stake (PoS) =====
Block validated by Idriss with PoS
Time taken for PoS: 0.0 seconds

Block v

### Result :

In the output, the differences between Proof of Work (PoW) and Proof of Stake (PoS) are highlighted:

#### Proof of Work (PoW)
- **Mining Process**: For each block, PoW attempts multiple hashes by adjusting the nonce until it finds a hash that matches the difficulty level (starting with four zeros).
- **Time Taken**: You can see the time taken for each PoW block (e.g., 0.37 seconds and 0.51 seconds) which reflects the computational work required to meet the difficulty target.
- **Hash Output**: Each mined block has a unique hash that fulfills the difficulty criteria.

#### Proof of Stake (PoS)
- **Validation Process**: Instead of mining, PoS selects a validator based on their stake. Here, "Mohammed" and "Hiba" are chosen to validate the blocks based on their stakes.
- **Time Taken**: The time for PoS is negligible (0.0 seconds), showing the efficiency of block addition as it skips the complex computations of PoW.
- **Validator Output**: The output specifies which validator approved each block.

#### Key Takeaways
- **Efficiency**: PoS is more time-efficient as it doesn’t require computational effort to solve a puzzle, unlike PoW.
- **Energy Usage**: PoS is energy-friendly since it doesn’t rely on extensive computations.
- **Security Model**: PoW relies on computational power, while PoS relies on validators’ stakes to maintain blockchain integrity.